In [ ]:
import json
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from gensim import corpora, models
import gensim
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder

# Read JSON

In [ ]:
# with open('fpmsdb.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)
    
with open('fpmsdoecedb.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
    
# print(data['features'][4115]['properties']['title'])

research_titles = []
for i in range(0, 565):
#     research_titles.append(data['features'][i]['properties']['title'])
    research_titles.append(data[i]['title'])

print(research_titles)

In [49]:
# download necessary resources from NLTK
# nltk.download('stopwords')
# nltk.download('wordnet')

# define stop words and lemmatizer
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

# preprocess function for text data
def preprocess(text):
    # tokenize the text and remove stop words and punctuation
    tokens = [word for word in nltk.word_tokenize(text.lower()) if word not in stop and word not in exclude]
    
    # extract collocations and add them to the token list
    bigram_measures = BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(2)  # only consider bigrams that occur at least twice
    collocations = finder.nbest(bigram_measures.pmi, 10)  # extract the top 10 collocations
    for collocation in collocations:
        if collocation[0] in tokens and collocation[1] in tokens:
            tokens.append('_'.join(collocation))
    
    # lemmatize the tokens
    normalized = " ".join(lemma.lemmatize(word) for word in tokens)
    return normalized

# preprocess the titles
titles_preprocessed = [preprocess(title) for title in research_titles]

# create a dictionary and corpus
dictionary = corpora.Dictionary([doc.split() for doc in titles_preprocessed])
corpus = [dictionary.doc2bow(doc.split()) for doc in titles_preprocessed]

# build the LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=dictionary,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto', # higher value => more concentrated topics
                                            per_word_topics=True)

# print the top 10 topics and their most significant words
topics = lda_model.show_topics(num_topics=20, num_words=3, formatted=False)
# for topic in topics:
#     print("Topic {}: {}".format(topic[0], ", ".join([word[0] for word in topic[1]])))
    
# categorize the research titles based on their highest probability topic
for i, title in enumerate(titles_preprocessed):
    bow = dictionary.doc2bow(preprocess(title).split())
    topic_probs = lda_model.get_document_topics(bow)
    topic_probs_sorted = sorted(topic_probs, key=lambda x: x[1], reverse=True)
    topic_num = topic_probs_sorted[0][0]
    print(research_titles[i])
    print("{} - Topic {}: {}".format(i+1, topic_num, ", ".join([word[0] for word in topics[topic_num][1]])))


Research Article Resiliency-Driven Approach of DC Microgrid Voltage Regulation Based on Droop Index Control for High Step-Up DC-DC Converter
1 - Topic 4: distributed, control, distribution
Effect of doubly fed induction generator based wind farms on the small signal stability of multi-machine power system
2 - Topic 18: model, evaluation, signal
Resiliency-Driven Approach of DC Microgrid Voltage Regulation Based on Droop Index Control for High Step-Up DC-DC Converter
3 - Topic 4: distributed, control, distribution
Probabilistic reliability evaluation of off-grid small hybrid solar PV-wind power system for the rural electrification in Nepal
4 - Topic 10: system, power, management
Reducing the cost of Rural Electrification: A key to improve the quality of life in rural areas in developing countries
5 - Topic 9: rural, energy, quality
Industrial customers' survey for outage cost valuation in a developing country
6 - Topic 2: cost, urban, developing
Estimation of global solar radiation usin

Recurrent Neural Network Based Forecasting of Food Production in Nepal
387 - Topic 0: network, using, analysis
StYLiD: Structured Information Sharing with User-defined Concepts
388 - Topic 8: based, engineering, sharing
Ontology based Job-Candidate Matching using Skill Sets
389 - Topic 0: network, using, analysis
大学研究者総覧 DB を対象とした横断検索のための概念抽出・統合の試み
390 - Topic 0: network, using, analysis
Road Accident News Information Extraction
391 - Topic 0: network, using, analysis
Community-Driven Consolidated Linked Data
392 - Topic 0: network, using, analysis
A Report on Linked Data
393 - Topic 0: network, using, analysis
A Novel Approach for Utilization and Enhancement of ICT using the Web
394 - Topic 0: network, using, analysis
Need of E-Recruitment System for Universities: Case of Pulchowk Campus, Nepal
395 - Topic 6: nepal, study, case
SocioBiblog: Enabling Communication on Bibliography with Semantic Blogging.
396 - Topic 17: computing, technology, international
Improvement of Complexity and 